In [9]:
import numpy as np
import pandas as pd
import memory_profiler
%load_ext memory_profiler


# Reading data via Pandas

In [60]:
%memit
%time
df = pd.read_csv(r'./NYPD_Complaint_Data_Historic.csv')

peak memory: 149.20 MiB, increment: 0.00 MiB


/home/krzysztof/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [63]:
df.head()

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,...,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX
0,522575447,08/29/2006,13:00:00,NaN,NaN,43.0,08/30/2006,578,HARRASSMENT 2,638.0,...,M,NaN,40.827414,-73.877946,"(40.827414051, -73.877945775)",PATROL BORO BRONX,NaN,25-44,BLACK HISPANIC,F
1,403507361,11/05/2006,11:00:00,11/05/2006,17:40:00,66.0,11/05/2006,107,BURGLARY,221.0,...,NaN,NaN,40.637097,-74.006105,"(40.637096864, -74.006105014)",PATROL BORO BKLYN SOUTH,NaN,45-64,ASIAN / PACIFIC ISLANDER,F
2,631420068,09/08/2006,23:30:00,09/09/2006,00:01:00,106.0,09/09/2006,347,INTOXICATED & IMPAIRED DRIVING,905.0,...,NaN,NaN,40.679260,-73.841500,"(40.679260229, -73.841500185)",PATROL BORO QUEENS SOUTH,NaN,NaN,UNKNOWN,E
3,995609899,12/13/2011,18:40:00,12/13/2011,18:49:00,79.0,12/13/2011,341,PETIT LARCENY,333.0,...,NaN,NaN,40.687402,-73.940369,"(40.687401619, -73.940369194)",PATROL BORO BKLYN NORTH,NaN,NaN,UNKNOWN,D
4,480667624,08/14/2009,04:20:00,NaN,NaN,30.0,08/14/2009,113,FORGERY,729.0,...,NaN,NaN,40.831576,-73.942983,"(40.831576128, -73.942982699)",PATROL BORO MAN NORTH,NaN,NaN,UNKNOWN,E


#### Checking number of missing rows

In [65]:
%memit
%time
df.isnull().sum();

peak memory: 4790.36 MiB, increment: 0.00 MiB
CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.91 µs


#### Counting number of crimes (basing on KY_CD)

In [68]:
%memit
%time
df.groupby('KY_CD').size();

peak memory: 4790.46 MiB, increment: 0.00 MiB
CPU times: user 0 ns, sys: 3 µs, total: 3 µs
Wall time: 7.63 µs


# Reading data via Dask

In [2]:
import dask
import dask.dataframe as dd


In [24]:
with open('./NYPD_Complaint_Data_Historic.csv') as file:
    header = file.readline()
headers = header.split(',')


In [25]:
dtypes = dict()
for header in headers:
    dtypes[header] = 'str'

In [28]:
%memit
%time
%timeit
df_dask = dd.read_csv(r'./NYPD_Complaint_Data_Historic.csv', dtype = dtypes)

peak memory: 584.56 MiB, increment: 0.00 MiB
CPU times: user 0 ns, sys: 4 µs, total: 4 µs
Wall time: 5.25 µs


### Checking number of missing rows

In [30]:
%memit
%time
%timeit
df_dask.isnull().sum().compute();

peak memory: 618.27 MiB, increment: -0.25 MiB
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs


### Counting number of crimes (basing on KY_CD)

In [32]:
%memit
%time
%timeit
df_dask.groupby('KY_CD').size();

peak memory: 642.43 MiB, increment: 0.00 MiB
CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.34 µs


# Dask as local cluster

In [34]:
from distributed import Client, LocalCluster

/home/krzysztof/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [35]:
localCluster = LocalCluster()
client = Client(localCluster)
client

Client Scheduler: tcp://127.0.0.1:45065 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.68 GB


In [36]:
%memit
%time
%timeit
df_dask = dd.read_csv(r'./NYPD_Complaint_Data_Historic.csv', dtype = dtypes)

peak memory: 644.68 MiB, increment: 0.20 MiB
CPU times: user 0 ns, sys: 3 µs, total: 3 µs
Wall time: 5.25 µs


In [37]:
%memit
%time
%timeit
df_dask.isnull().sum().compute();

peak memory: 645.59 MiB, increment: 0.14 MiB
CPU times: user 0 ns, sys: 2 µs, total: 2 µs
Wall time: 8.58 µs


In [38]:
%memit
%time
%timeit
df_dask.groupby('KY_CD').size();

peak memory: 646.48 MiB, increment: 0.09 MiB
CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.25 µs
